In [93]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time
import datetime

import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from scipy.signal import savgol_filter
import statsmodels.formula.api as smf

In [94]:
frida_products = get_fresh_query_result("https://internal-redash.federate.frubana.com/",118629,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [95]:
df_check = get_fresh_query_result("https://internal-redash.federate.frubana.com/",165821,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [96]:
#df_margin = get_fresh_query_result("https://internal-redash.federate.frubana.com/",144521,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

In [97]:
df_margin = df_check.loc[~(df_check.fit_for_discount == 'FALSE'),['card_id','margin']].groupby(['card_id']).margin.mean().reset_index()

In [98]:
custom = pd.read_csv("push customers - customers.csv")

In [99]:
print(f"HOLA {city}")

HOLA BHZ


In [135]:
city = 'BHZ'
clientes = custom.loc[custom.city == city].customer_id.unique().tolist()
dias = 0
#year-month-day
todays_date = datetime.datetime.today().strftime('%Y-%m-%d')
#non_buyers_date = '2022-08-26' #1 months
customer_date_desde_query = (datetime.datetime.today() - datetime.timedelta(days=30)).strftime('%Y-%m-%d') #1 months

customer_date_desde = (datetime.datetime.today() - datetime.timedelta(days=200))#.strftime('%Y-%m-%d') #1 months

In [136]:
custom

,customer_id,city
0,347777930,BHZ
1,289673641,BHZ
2,347537418,BHZ
3,347701755,BHZ
4,347265892,BHZ
...,...,...
6113,63444896,SPO
6114,346894277,SPO
6115,203963042,SPO
6116,80983679,SPO


In [137]:
len(clientes)

551

In [138]:
query = """
-- 93058
-- Se toma como base el query 2718
WITH 
ful AS (

SELECT DISTINCT
    bo.order_id,
    bo.submit_date
    
FROM postgres_broadleaf_federate."broadleaf.blc_order" bo
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group" bfg        ON bfg.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order" bfo        ON bfo.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group" ffg         ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order" fo                      ON fo.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_payment" bop            ON bop.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site" s                       ON s.site_id = bo.site_disc
LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type" fot                ON fot.fb_order_type_id=fo.fb_order_type_id

WHERE fo.fb_order_status_id IN (1,6,7,8)
    --AND extract(year from bo.submit_date::date) > 2021
    --AND extract(month from bo.submit_date::date) = 6--2021
    AND bo.order_status = 'SUBMITTED'
    AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
    AND bop.archived = 'N'
    AND (fot.name IS NULL OR fot.name <> 'REFUND')
    AND s.site_identifier_value = '{city}' --NOT IN ('MTY','PBC','GDL')
    AND bo.customer_id IN {clientes}
)--,

--more AS (
SELECT DISTINCT
    s.site_identifier_value AS city,
    bo.customer_id,
    --DATE(bo.submit_date) AS submit_date,
    bs.addl_product_id AS card_id,
    bs2.name,
    COUNT(DISTINCT bo.order_id) AS frequency,
    SUM(boipd.quantity * foi.step_unit * boi.sale_price)*1.0 AS gmv_usd,
    SUM(boipd.quantity*foi.step_unit) AS demand

FROM postgres_broadleaf_federate."broadleaf.blc_order" bo
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                     s       ON s.site_id = bo.site_disc
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"               boi     ON boi.order_id=bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order_item"                foi     ON boi.order_item_id= foi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                     fo      ON fo.order_id = bo.order_id
INNER JOIN ful                                                                          ON ful.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_discrete_order_item"      bdoi    ON bdoi.order_item_id = boi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                      bs      ON bs.sku_id = bdoi.sku_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"                  bp      ON bs.addl_product_id = bp.product_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                      bs2     ON bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                 bcat    ON bcat.category_id = bp.default_category_id 
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category_xref"            bcx     ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                 bcat2   ON bcx.category_id = bcat2.category_id
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_price_dtl"      boipd   ON boipd.order_item_id=boi.order_item_id
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_dtl_adj"        boida   ON boida.order_item_price_dtl_id=boipd.order_item_price_dtl_id

WHERE fb_order_status_id IN (1,6,7,8)
    AND bo.order_status = 'SUBMITTED'
   -- FIX SUPER DESCUENTOS
    AND bcat2.category_id not in ('110873','-1000','100768','100765','100815') --ids de super descuentos en cada país
    AND bcat.name <> 'Oferton Frubana' AND bcat2.name <> 'Oferton Frubana'
GROUP BY 1,2,3,4
""".format(clientes=tuple(set(clientes)), city = city)

data_ventas_aux = read_connection_data_warehouse.runQuery(query)
data_ventas_aux[["frequency", "gmv_usd","demand"]] = data_ventas_aux[["frequency", "gmv_usd","demand"]].astype('float32')
#data_ventas_aux['submit_date'] = pd.to_datetime(data_ventas_aux['submit_date']) 

In [139]:
# Assuming your DataFrames are named 'df1' and 'df2'
data_ventas_aux = pd.merge(data_ventas_aux, df_margin[['margin','card_id']], left_on='card_id', right_on='card_id', how='inner')

In [140]:
data_ventas_aux = data_ventas_aux.loc[~data_ventas_aux.card_id.isin([55148,80182,-201566,-201623])].reset_index(drop=True)

In [141]:
# Optional to clean the EagleEye
print(len(data_ventas_aux))

15309


In [142]:
data_ventas_aux['cash_margin'] = data_ventas_aux.gmv_usd * data_ventas_aux.margin

In [143]:
# Assuming your DataFrame is named 'df'
grouped_df = data_ventas_aux.groupby('customer_id').apply(lambda x: x.nlargest(6, ['frequency', 'gmv_usd'])).reset_index(drop=True)

result_df = grouped_df.groupby('customer_id').agg({'card_id': list, 'name': list, 'frequency': list, 'gmv_usd': np.sum, 'cash_margin': np.sum}).reset_index()

result_df["margin"] = 100.00 * result_df.cash_margin / result_df.gmv_usd
result_df['margin'] = np.where(0.85*result_df.margin > 12.5, 12.5,0.85*result_df.margin)
result_df['margin'] = result_df['margin'].apply(lambda x: round(x, 2))
result_df['offer_name'] = ["SH_GRADS_"+city+"_"+todays_date+"_"+str(customer) for customer in result_df.customer_id]

result_df = result_df.loc[result_df.margin > 0].copy()

In [144]:
result_df

,customer_id,card_id,name,frequency,gmv_usd,cash_margin,margin,offer_name
0,60763282,"[225109, 431107, 253074, 274968, 447725, 237360]","[Açúcar Cristal Laçucar 5Kg, Queijo Mussarela ...","[34.0, 9.0, 9.0, 7.0, 6.0, 5.0]",45092.082031,4592.095019,8.66,SH_GRADS_BHZ_2023-09-29_60763282
1,63963775,"[202911, 202728, 237366, 253074, 211237, 237371]","[Óleo de Soja Veleiro 900ml, Óleo de Soja ABC ...","[26.0, 7.0, 7.0, 5.0, 4.0, 3.0]",10900.500000,705.753231,5.50,SH_GRADS_BHZ_2023-09-29_63963775
2,64575706,"[207923, 443568, 443567, 362369, 202728, 202911]","[Néctar Laranja Tial 330ml, Água Aguaí Sem Gás...","[8.0, 7.0, 5.0, 3.0, 2.0, 1.0]",1384.959961,241.311617,12.50,SH_GRADS_BHZ_2023-09-29_64575706
3,64723697,"[451681, 202728, 202911, 207763, 443568, 248819]","[Ovos Brancos Grandes O Rei do Ovo, Óleo de So...","[2.0, 1.0, 1.0, 1.0, 1.0, 1.0]",679.280029,123.384899,12.50,SH_GRADS_BHZ_2023-09-29_64723697
4,64764282,"[237363, 216309, 225109, 202911, 237362, 250063]","[Café Tradicional Pouch 3 Corações 500g, Açúca...","[57.0, 51.0, 40.0, 33.0, 20.0, 14.0]",22245.791016,2479.745727,9.47,SH_GRADS_BHZ_2023-09-29_64764282
...,...,...,...,...,...,...,...,...
546,351945926,"[409534, 340361, 451672, 340009, 202660, 211331]","[Filé de Peito de Frango - Avivar, Guaraná Ant...","[5.0, 3.0, 3.0, 2.0, 2.0, 2.0]",785.400024,86.142253,9.32,SH_GRADS_BHZ_2023-09-29_351945926
547,351949441,"[250063, 216312, 202911, 271580, 447725, 347713]","[Cerveja Heineken 473ml, Feijão Preto Cocal 1K...","[4.0, 2.0, 1.0, 1.0, 1.0, 1.0]",1377.160034,152.105623,9.39,SH_GRADS_BHZ_2023-09-29_351949441
548,352163796,"[202911, 274964, 440112, 207921, 207918, 274969]","[Óleo de Soja Veleiro 900ml, Queijo Mussarela ...","[6.0, 5.0, 5.0, 3.0, 2.0, 1.0]",2551.800049,216.732644,7.22,SH_GRADS_BHZ_2023-09-29_352163796
549,352234536,"[225109, 340009, 202911, 420567, 216309, 443568]","[Açúcar Cristal Laçucar 5Kg, Coca-Cola 200ml, ...","[3.0, 2.0, 2.0, 1.0, 1.0, 1.0]",1358.910034,109.346241,6.84,SH_GRADS_BHZ_2023-09-29_352234536


In [145]:
# Step 1: Convert 'card_id' column from lists to individual values
df_exploded = result_df.explode('card_id')

# Step 2: Count the occurrences of each card ID
card_id_counts = df_exploded['card_id'].value_counts().reset_index()

# Rename columns and set 'card_id' as the index
card_id_counts.rename(columns={'index': 'card_id', 'card_id': 'frequency'}, inplace=True)

# Calculate cumulative percentage and percentage over the total
card_id_counts['cumulative_percentage'] = card_id_counts['frequency'].cumsum() / card_id_counts['frequency'].sum()
card_id_counts['percentage_over_total'] = card_id_counts['frequency'] / card_id_counts['frequency'].sum()

# Multiply by 100 and round to two decimal places
card_id_counts['cumulative_percentage'] = (card_id_counts['cumulative_percentage'] * 100).round(2)
card_id_counts['percentage_over_total'] = (card_id_counts['percentage_over_total'] * 100).round(2)

In [146]:
df_demand =  pd.merge(card_id_counts, data_ventas_aux[['card_id','name']], left_on='card_id', right_on='card_id', how='inner')
df_demand.drop_duplicates(subset=['card_id'], keep='first', inplace=True)

In [147]:
def concatenate_skus(card_id_list):
    result = []
    for sku in card_id_list:
        try:
            sku_value = frida_products.loc[frida_products['id_tarjeta'] == sku, 'skus'].values[0]
            result.append(sku_value)
        except IndexError:
            result.append('')
    return '#'.join(result)

In [148]:
from sklearn.model_selection import train_test_split 

def create_groups(offer):
    
    #customers = offer.customer_id.unique()
    customers = offer.customer_id.unique()
    print("Total length: ", len(customers))
    customer_control, customer_test = train_test_split(customers, test_size = 0.50)
    
    alpha=0.01
    aux=0
    while abs(1-(offer[offer.customer_id.isin(customer_control)]['gmv_usd'].mean()/offer[offer.customer_id.isin(customer_test)]['gmv_usd'].mean())) > alpha:
        customer_control, customer_test = train_test_split(customers, test_size = 0.50)
        
        aux+=1
        if aux == 1000:
            alpha+=0.01
            aux=0
        
    return customer_control.tolist(), customer_test.tolist()

In [149]:
control,test = create_groups(result_df)

Total length:  551


In [150]:
final_df = pd.concat([
    pd.DataFrame({'group': 'control', 'customer_id': control}),
    pd.DataFrame({'group': 'test', 'customer_id': test})
], ignore_index=True)

final_df['city'] = city
final_df['todays_date'] = todays_date

In [151]:
final_df.group.value_counts()

test       276
control    275
Name: group, dtype: int64

In [152]:
final_df.to_csv("SH_GRADS"+todays_date+city+"GROUPS_9a16.csv", index=False)

In [153]:
global_segment_list = []

for i in result_df.loc[result_df.customer_id.isin(test)].values:
    global_segment_list.append({'name':i[-1], 'customersIds': [i[0]],"expiresAt": (datetime.datetime.today() + datetime.timedelta(days=(dias+2))).strftime('%Y-%m-%d')})
    
result_df = result_df.loc[result_df.customer_id.isin(test)].reset_index(drop=True).copy()

In [154]:
result_df

,customer_id,card_id,name,frequency,gmv_usd,cash_margin,margin,offer_name
0,63963775,"[202911, 202728, 237366, 253074, 211237, 237371]","[Óleo de Soja Veleiro 900ml, Óleo de Soja ABC ...","[26.0, 7.0, 7.0, 5.0, 4.0, 3.0]",10900.500000,705.753231,5.50,SH_GRADS_BHZ_2023-09-29_63963775
1,64764282,"[237363, 216309, 225109, 202911, 237362, 250063]","[Café Tradicional Pouch 3 Corações 500g, Açúca...","[57.0, 51.0, 40.0, 33.0, 20.0, 14.0]",22245.791016,2479.745727,9.47,SH_GRADS_BHZ_2023-09-29_64764282
2,65024696,"[202911, 253074, 447725, 202728, 207763, 237371]","[Óleo de Soja Veleiro 900ml, Leite Integral Po...","[40.0, 10.0, 9.0, 5.0, 3.0, 2.0]",30916.271484,1766.838052,4.86,SH_GRADS_BHZ_2023-09-29_65024696
3,66378545,"[202911, 216311, 216312, 368614, 274596, 255159]","[Óleo de Soja Veleiro 900ml, Feijão Carioca Pa...","[35.0, 32.0, 23.0, 20.0, 15.0, 6.0]",11589.740234,1817.532781,12.50,SH_GRADS_BHZ_2023-09-29_66378545
4,67563549,"[202911, 216311, 207763, 211561, 202677, 202728]","[Óleo de Soja Veleiro 900ml, Feijão Carioca Pa...","[28.0, 14.0, 11.0, 11.0, 11.0, 10.0]",14786.998047,1181.675659,6.79,SH_GRADS_BHZ_2023-09-29_67563549
...,...,...,...,...,...,...,...,...
271,351339395,"[207912, 202911, 443568, 340009, 460912, 341960]","[Coca-Cola 350ml, Óleo de Soja Veleiro 900ml, ...","[5.0, 3.0, 3.0, 2.0, 2.0, 2.0]",391.649994,56.804045,12.33,SH_GRADS_BHZ_2023-09-29_351339395
272,351879351,"[447725, 202911, 274964, 211573, 211576, 460236]","[Leite Integral 1 Litro - Ita, Óleo de Soja Ve...","[2.0, 1.0, 1.0, 1.0, 1.0, 1.0]",658.369995,73.757486,9.52,SH_GRADS_BHZ_2023-09-29_351879351
273,351949441,"[250063, 216312, 202911, 271580, 447725, 347713]","[Cerveja Heineken 473ml, Feijão Preto Cocal 1K...","[4.0, 2.0, 1.0, 1.0, 1.0, 1.0]",1377.160034,152.105623,9.39,SH_GRADS_BHZ_2023-09-29_351949441
274,352234536,"[225109, 340009, 202911, 420567, 216309, 443568]","[Açúcar Cristal Laçucar 5Kg, Coca-Cola 200ml, ...","[3.0, 2.0, 2.0, 1.0, 1.0, 1.0]",1358.910034,109.346241,6.84,SH_GRADS_BHZ_2023-09-29_352234536


In [155]:
csv_file = {} 
csv_file['offer_name'] = result_df.offer_name
csv_file['offer_description'] = None
csv_file['discount'] = result_df.margin
csv_file['automatically_consider_offer'] = "true"
csv_file['start_date'] = (datetime.datetime.today() + datetime.timedelta(days=dias)).strftime('%d-%m-%Y')
csv_file['end_date'] = (datetime.datetime.today() + datetime.timedelta(days=(dias+2))).strftime('%d-%m-%Y')
csv_file['max_uses_per_order'] = 120
csv_file['max_uses_per_customer'] = 2
csv_file['customer_segment_id'] = [1]*len(result_df.offer_name) #[None]
csv_file['sku_id'] = result_df['card_id'].apply(concatenate_skus)

global_offer_csv = pd.DataFrame(csv_file) #, index=[0])

In [156]:
global_offer_csv.head()

,offer_name,offer_description,discount,automatically_consider_offer,start_date,end_date,max_uses_per_order,max_uses_per_customer,customer_segment_id,sku_id
0,SH_GRADS_BHZ_2023-09-29_63963775,None,5.50,true,29-09-2023,01-10-2023,120,2,1,252145#251946#306010#328373#263792#306025
1,SH_GRADS_BHZ_2023-09-29_64764282,None,9.47,true,29-09-2023,01-10-2023,120,2,1,306001#269739#283189#252145#305998#324003#3240...
2,SH_GRADS_BHZ_2023-09-29_65024696,None,4.86,true,29-09-2023,01-10-2023,120,2,1,252145#328373#621871#251946#257901#306025
3,SH_GRADS_BHZ_2023-09-29_66378545,None,12.50,true,29-09-2023,01-10-2023,120,2,1,252145#269745#269748#503109#356539#331189
4,SH_GRADS_BHZ_2023-09-29_67563549,None,6.79,true,29-09-2023,01-10-2023,120,2,1,252145#269745#257901#263741#251843#251946


In [157]:
# Replace two or three '#' with one '#'
global_offer_csv['sku_id'] = global_offer_csv['sku_id'].str.replace('###', '#')
global_offer_csv['sku_id'] = global_offer_csv['sku_id'].str.replace('##', '#')
    
# Replace '#' at the beginning with an empty string
global_offer_csv['sku_id'] = global_offer_csv['sku_id'].str.replace('^#', '', regex=True)

# Remove rows where the string is empty
global_offer_csv = global_offer_csv.loc[global_offer_csv['sku_id'] != ""]

In [158]:
global_offer_csv

,offer_name,offer_description,discount,automatically_consider_offer,start_date,end_date,max_uses_per_order,max_uses_per_customer,customer_segment_id,sku_id
0,SH_GRADS_BHZ_2023-09-29_63963775,None,5.50,true,29-09-2023,01-10-2023,120,2,1,252145#251946#306010#328373#263792#306025
1,SH_GRADS_BHZ_2023-09-29_64764282,None,9.47,true,29-09-2023,01-10-2023,120,2,1,306001#269739#283189#252145#305998#324003#3240...
2,SH_GRADS_BHZ_2023-09-29_65024696,None,4.86,true,29-09-2023,01-10-2023,120,2,1,252145#328373#621871#251946#257901#306025
3,SH_GRADS_BHZ_2023-09-29_66378545,None,12.50,true,29-09-2023,01-10-2023,120,2,1,252145#269745#269748#503109#356539#331189
4,SH_GRADS_BHZ_2023-09-29_67563549,None,6.79,true,29-09-2023,01-10-2023,120,2,1,252145#269745#257901#263741#251843#251946
...,...,...,...,...,...,...,...,...,...,...
271,SH_GRADS_BHZ_2023-09-29_351339395,None,12.33,true,29-09-2023,01-10-2023,120,2,1,258148#258149#304090#252145#616555#453392#4533...
272,SH_GRADS_BHZ_2023-09-29_351879351,None,9.52,true,29-09-2023,01-10-2023,120,2,1,621871#252145#356859#263783#263945#637044
273,SH_GRADS_BHZ_2023-09-29_351949441,None,9.39,true,29-09-2023,01-10-2023,120,2,1,324003#324005#324004#269748#252145#352806#6218...
274,SH_GRADS_BHZ_2023-09-29_352234536,None,6.84,true,29-09-2023,01-10-2023,120,2,1,283189#453392#453391#453390#252145#586755#2697...


In [159]:
import requests
import json

url = f"https://{city}.frubana.com/api/v1/segments/create?api-key=00_growth-team-zzz-qqq_001"

payload = json.dumps(global_segment_list)


headers = {
  'Cookie': 'AWSALB=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALBCORS=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALB=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI; AWSALBCORS=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

In [160]:
response.text

'Customer segments were created: [364905-SH_GRADS_BHZ_2023-09-29_63963775, 364904-SH_GRADS_BHZ_2023-09-29_64764282, 364903-SH_GRADS_BHZ_2023-09-29_65024696, 364906-SH_GRADS_BHZ_2023-09-29_66378545, 364907-SH_GRADS_BHZ_2023-09-29_67563549, 364908-SH_GRADS_BHZ_2023-09-29_67570915, 364909-SH_GRADS_BHZ_2023-09-29_68205926, 364910-SH_GRADS_BHZ_2023-09-29_69575563, 364912-SH_GRADS_BHZ_2023-09-29_69826083, 364911-SH_GRADS_BHZ_2023-09-29_70588996, 364913-SH_GRADS_BHZ_2023-09-29_71076350, 364914-SH_GRADS_BHZ_2023-09-29_71475258, 364915-SH_GRADS_BHZ_2023-09-29_72583536, 364916-SH_GRADS_BHZ_2023-09-29_72593259, 364918-SH_GRADS_BHZ_2023-09-29_73115546, 364917-SH_GRADS_BHZ_2023-09-29_73466680, 364919-SH_GRADS_BHZ_2023-09-29_74327033, 364920-SH_GRADS_BHZ_2023-09-29_74909773, 364921-SH_GRADS_BHZ_2023-09-29_75471800, 364922-SH_GRADS_BHZ_2023-09-29_75637242, 364923-SH_GRADS_BHZ_2023-09-29_75958144, 364925-SH_GRADS_BHZ_2023-09-29_76721051, 364926-SH_GRADS_BHZ_2023-09-29_77647924, 364924-SH_GRADS_BHZ_202

In [161]:
sgt = [(int(x[:6]),x[7:]) for x in list(map(str.strip, response.text[32:].strip('][').replace('"', '').split(',')))]

In [162]:
for i in global_offer_csv.offer_name:
    for j,z in sgt:
        if z==i:
            global_offer_csv.loc[global_offer_csv.offer_name == i, ('customer_segment_id')] = j
            break

In [163]:
global_offer_csv.head()

,offer_name,offer_description,discount,automatically_consider_offer,start_date,end_date,max_uses_per_order,max_uses_per_customer,customer_segment_id,sku_id
0,SH_GRADS_BHZ_2023-09-29_63963775,None,5.50,true,29-09-2023,01-10-2023,120,2,364905,252145#251946#306010#328373#263792#306025
1,SH_GRADS_BHZ_2023-09-29_64764282,None,9.47,true,29-09-2023,01-10-2023,120,2,364904,306001#269739#283189#252145#305998#324003#3240...
2,SH_GRADS_BHZ_2023-09-29_65024696,None,4.86,true,29-09-2023,01-10-2023,120,2,364903,252145#328373#621871#251946#257901#306025
3,SH_GRADS_BHZ_2023-09-29_66378545,None,12.50,true,29-09-2023,01-10-2023,120,2,364906,252145#269745#269748#503109#356539#331189
4,SH_GRADS_BHZ_2023-09-29_67563549,None,6.79,true,29-09-2023,01-10-2023,120,2,364907,252145#269745#257901#263741#251843#251946


In [164]:
#//--------------------------------------------
#//Exporting single file data
global_offer_csv.to_csv(f'{todays_date}_SH_FARMING_{city}_OFFER_csv.csv', index = False)

In [165]:
#result_df.to_excel(f'SD/export_data/Below_Media/{city}/{todays_date}_SH_RESULTS_FARMING_{city}_OFFER_csv.xlsx', index = False)